In [1]:
import pandas as pd

In [2]:
xl = pd.ExcelFile('./[과제3] 식당 메뉴 및 식사인원.xlsx')

In [3]:
xl.sheet_names

['설명', '식사인원', '메뉴']

In [4]:
menu = xl.parse('메뉴')
menu.info()
menu.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24860 entries, 1 to 24860
Data columns (total 3 columns):
일자      24860 non-null int64
식사명     24860 non-null object
식사내용    24860 non-null object
dtypes: int64(1), object(2)
memory usage: 776.9+ KB


,일자,식사명,식사내용
1,20030301,아침식사,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥"
2,20030301,아침식사2,"딸기잼(중),비엔나구이,스크램블에그(경양식),야채샐러드,크림스프(경양식),토스트&모닝빵"
3,20030301,점심식사,"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수"
4,20030301,저녁식사,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프"
5,20030302,아침식사,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침"


In [5]:
menu['split_content'] = menu['식사내용'].apply(lambda x: x.split(','))
menu['content_len'] = menu['split_content'].apply(lambda x: len(x))

In [6]:
menu['content_len'].value_counts()

8     9394
7     7259
6     4730
10    1155
9      896
5      635
4      431
3      244
11      55
13      15
12      14
1        6
15       6
2        5
14       5
16       4
17       3
19       2
18       1
Name: content_len, dtype: int64

In [7]:
menu['doc_tag'] = menu.index.astype(str).tolist()

In [8]:
from gensim.models import Doc2Vec

In [9]:
input_list = menu['split_content'].tolist()

In [10]:
from collections import namedtuple

In [11]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [12]:
tagged_train_docs = [TaggedDocument(d, c) for d, c in menu[['split_content','doc_tag']].values]

In [47]:
%%time
model = Doc2Vec(dm=0,dbow_words=1,hs=1 ,window=5,vector_size=100,min_count=0,negative=10,iter=30)

CPU times: user 514 µs, sys: 175 µs, total: 689 µs
Wall time: 653 µs


/Users/1003874/anaconda/envs/gcp-vision/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")


In [48]:
model.build_vocab(tagged_train_docs)

In [49]:
print(str(model))

Doc2Vec(dbow+w,d100,n10,hs,w5,s0.001,t3)


In [50]:
for epoch in range(10):
    model.train(tagged_train_docs, total_examples=model.corpus_count, epochs=3)

In [55]:
model.wv.most_similar('돈까스')

[('치즈함박스테이크&데미', 0.8897284865379333),
 ('치즈돈까스', 0.8770416975021362),
 ('함박스테이크', 0.873145580291748),
 ('모듬(등심.생선)', 0.8466408252716064),
 ('치킨함박스테이크', 0.825142502784729),
 ('치킨까스&데미소스', 0.7687565088272095),
 ('조랭이함박', 0.6851581335067749),
 ('치킨까스&파채(칠리소스)', 0.649042546749115),
 ('모듬커틀렛(치)', 0.6406075954437256),
 ('돈까스&불맛소스', 0.6230874061584473)]

In [56]:
X_train = [model.infer_vector(doc.words) for doc in tagged_train_docs]
y_train = [doc.tags for doc in tagged_train_docs]

In [57]:
X_train_np = np.asarray(X_train)
X_train_np.shape

(24860, 100)

In [58]:
model.save('./food_doc.model')